In [9]:
import polars as pl
import polars.selectors as cs
import pandas as pd

In [10]:
def to_polars(df):
    return pl.DataFrame(df)

## 569 

In [11]:
data = [[1, 'A', 2341], [2, 'A', 341], [3, 'A', 15], [4, 'A', 15314], [5, 'A', 451], [6, 'A', 513], [7, 'B', 15], [8, 'B', 13], [9, 'B', 1154], [10, 'B', 1345], [11, 'B', 1221], [12, 'B', 234], [13, 'C', 2345], [14, 'C', 2645], [15, 'C', 2645], [16, 'C', 2652], [17, 'C', 65]]
employee = pd.DataFrame(data, columns=['id', 'company', 'salary']).astype({'id':'Int64', 'company':'object', 'salary':'Int64'})

In [12]:
employee=pl.DataFrame(employee)

In [13]:
(
    employee
    .with_columns(
        pl.last().rank(method="ordinal").over("company").alias("rank"),
        pl.first().len().over("company").alias('len') /2,
    )
    .sort(by=["company","salary", "id"])
    .filter(
        cs.last()+1>=pl.col('rank'), cs.last()<= pl.col("rank") # find median salary for each company
    )
    # .select(cs.by_index(0,1,2))
)

id,company,salary,rank,len
i64,str,i64,u32,f64
5,"""A""",451,3,3.0
6,"""A""",513,4,3.0
12,"""B""",234,3,3.0
9,"""B""",1154,4,3.0
14,"""C""",2645,3,2.5


## 571

In [14]:
data = [[0, 7], [1, 1], [2, 3], [3, 1]]
numbers = pd.DataFrame(data, columns=['num', 'frequency']).astype({'num':'Int64', 'frequency':'Int64'}).pipe(to_polars)

In [15]:
numbers

num,frequency
i64,i64
0,7
1,1
2,3
3,1


In [16]:
import numpy as np

nn = numbers.to_numpy()
arr = np.repeat(nn[:, 0], nn[:, 1])
np.quantile(arr, 0.5)

0.0